# Recommending movies using Truncated SVD
#### There are latent features in the reconstructed matrix showing a correlation with the user ratings for the rating prediction.

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() 
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn import preprocessing 
from sklearn.preprocessing import OneHotEncoder
from sklearn.decomposition import TruncatedSVD
import math

In [4]:
def distance(p1,p2):
    distance = math.sqrt( ((p1[0]-p2[0])**2)+((p1[1]-p2[1])**2) )
    return distance 

def corr_percentage(df, set1,set2):
    test = df[df['Set'] == set1]
    #print(test.head(10))
    confirm = df[df.UserID.isin(test.UserID.unique())]
    #print(confirm.head(10))
    confirm = confirm[confirm['Set']== set2]
    return len(confirm.UserID.unique())/len(test.UserID.unique())


def correlation_fun(df):
    """
    df -> dataframe: the dataframe to be analyzed by svd recommendation engine
          with 3 columns, UserID, Set, Rating
    """
    df_user_le = pd.DataFrame(df["UserID"].tolist(), columns=['user'])
    le = preprocessing.LabelEncoder()
    df_user_le['user_le'] = le.fit_transform(df_user_le['user'])
    df_set_le = pd.DataFrame(df["Set"].tolist(), columns=['set'])
    df_set_le['set_le'] = le.fit_transform(df_set_le['set'])

    user_ids = np.array(df_user_le['user_le'].tolist())
    set_ids = np.array(df_set_le['set_le'].tolist())
    rating_ids = np.array(df["Rating"].tolist())
    n_user = df['UserID'].nunique()
    n_set = df['Set'].nunique()
    print ('num of users:',df['UserID'].nunique())
    print ('num of sets:',df['Set'].nunique())
    print (rating_ids.shape[0])
    
    graph = tf.Graph()


    embedding_size = 30

    lr = 0.0001
    reg = 0.01

    with graph.as_default():
        user = tf.placeholder(tf.int32, name="user_id") 
        sets = tf.placeholder(tf.int32, name="set") 
        rating = tf.placeholder(tf.float32, name="rating") 

        set_embedding = tf.Variable(tf.truncated_normal([n_set, embedding_size], stddev=0.02, mean=0.) ,name="set_embedding")
        user_embedding = tf.Variable(tf.truncated_normal([n_user, embedding_size], stddev=0.02, mean=0.) ,name="user_embedding")

        set_bias_embedding = tf.Variable(tf.truncated_normal([n_set], stddev=0.02, mean=0.) ,name="set_bias_embedding")
        user_bias_embedding = tf.Variable(tf.truncated_normal([n_user], stddev=0.02, mean=0.) ,name="user_bias_embedding")


        global_bias = tf.Variable(tf.truncated_normal([], stddev=0.02, mean=0.) ,name="global_bias")

        u = tf.nn.embedding_lookup(user_embedding, user)
        m = tf.nn.embedding_lookup(set_embedding, sets)

        u_bias = tf.nn.embedding_lookup(user_bias_embedding, user)
        m_bias = tf.nn.embedding_lookup(set_bias_embedding, sets)


        predicted_rating = tf.reduce_sum(tf.multiply(u, m), 1) + u_bias + m_bias + global_bias

        rmse = tf.sqrt(tf.reduce_mean(tf.square(predicted_rating - rating))) # RMSE
        cost = tf.nn.l2_loss(predicted_rating - rating)
        regularization = reg * (tf.nn.l2_loss(set_embedding) + tf.nn.l2_loss(user_embedding)
                                + tf.nn.l2_loss(set_bias_embedding) + tf.nn.l2_loss(user_bias_embedding))

        loss = cost + regularization

        optimizer = tf.train.AdamOptimizer(lr).minimize(loss)
        
    # 2 numbers you can play with to fit your data more
    batch_size = 5
    n_epoch = 30

    print(rating_ids.shape[0])
    with tf.Session(graph=graph) as sess:
        tf.initialize_all_variables().run()
        for _ in range(n_epoch):
            for start in range(0, rating_ids.shape[0] - batch_size, batch_size):
                end = start + batch_size
                _, cost_value = sess.run([optimizer, rmse], feed_dict={user: user_ids[start:end],
                                                      sets: set_ids[start: end],
                                                      rating: rating_ids[start: end]})

#             print ("RMSE", cost_value)
        embeddings = set_embedding.eval()
    

    df_set_fea_frame = pd.pivot_table(df, values='Rating', index=['Set'],columns=['UserID'], fill_value=0)
    df_set_fea = pd.pivot_table(df, values='Rating', index=['Set'],columns=['UserID'], fill_value=0).to_numpy()

    print ('{0}x{1} user by movie matrix'.format(*df_mv_fea.shape))
    

    df_rating1 = df.sample(n=df.shape[0], replace=False, random_state=1)
    R = pd.pivot_table(df, values='Rating', index=['UserID'],columns=['Set'], fill_value=0).to_numpy()
    # R = df_rating_mv[['userId','rating']].values
    print ('{0}x{1} user by user matrix'.format(*R.shape))


    from sklearn.decomposition import TruncatedSVD
    import math
    set_train_num = math.ceil(n_set)
    user_train_num = math.ceil(n_user)
    train_user = R[:, :]
    #test_user = R[math.ceil(R.shape[0]*0.8):, math.ceil(R.shape[1]*0.8):]
    train_set = df_set_fea[:, :]
    #test_mv = df_mv_fea[math.ceil(df_mv_fea.shape[0]*0.8):, math.ceil(df_mv_fea.shape[1]*0.8):]

    set_svd = TruncatedSVD(n_components=100)
    set_features = set_svd.fit_transform(train_mv)

    print ("set_features.shape = {0}".format(set_features.shape))


    user_svd = TruncatedSVD(n_components=100)
    user_features = user_svd.fit_transform(train_user)

    print ("user_features.shape = {0}".format(user_features.shape))
    
    from sklearn.manifold import TSNE
    import matplotlib.pyplot as plt
    %matplotlib inline

    tsne = TSNE(perplexity=5, n_components=2, init="pca", n_iter=5000)
    plot_only = 50
    coords = tsne.fit_transform(embeddings[:,:])
    print(len(coords))
    #print(set_features)
    plt.figure(figsize=(18, 18))
#     labels = [df.iloc[i].Set for i in range(plot_only)]
    labels = df.Set.to_list()
    #print(labels)
    already_cover = set()
    set_dict={}
    set_res = {}
    print("start to sort the results....")
    index = 0
    for i, label in enumerate(labels):
        if not label in set_dict:
#             x, y = coords[i, :]
            set_dict[label] = coords[index, :]
            index = index+1
#             print(label, coords[i, :])
        
    for k,v in set_dict.items():
        min_distance = float('inf')
        for k1,v1 in set_dict.items():
            dis = distance(v,v1)
            if (dis < min_distance) and (not dis == 0):
                set_res[k] = k1 
                min_distance = dis
            else:
                continue
    print(set_res)
    return set_res
        
#         plt.scatter(x, y)
#         plt.annotate(label,
#                      xy=(x, y),
#                      xytext=(10, 4),
#                      textcoords="offset points",
#                      ha="right",
#                      va="bottom")

    #plt.show()
    